In [2]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b")

In [4]:
# llm.generate("The quick brown fox jumps over the lazy dog")

In [5]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")
graph = graph_builder.compile()

def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1])

while True:
    try:
        user_input = input("User: ")
        print("User: " + user_input)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = input()
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: how many r are their in straberry?
Assistant: There is no such fruit as a "straberry." However, I think you might be thinking of an "blueberry" or possibly an "raspberry." If you're looking for the number of R's in those words, here they are:

* Blueberry: 2 (B-R-B)
* Raspberry: 3 (R-A-S-P-B-E-R-Y)
User: I mean strawberry?
Assistant: I think there might be some confusion. This conversation just started, and we haven't discussed anything yet, including strawberries. If you'd like to start a conversation about strawberries or any other topic, I'm here to help!
User: I mean how many r are their in strawberry?
Assistant: The number of R's in "strawberry" can vary depending on the font and style of the word, but I'll give you an approximate count.

In general, there are two or three R's in the word "strawberry."
User: 
Assistant: Can I help you with something or would you like to start a conversation?
User: q
Goodbye!
